In [1]:
# Library
from base import S3_SELECTED_CLUSTERS
from pandas import read_csv, DataFrame
from numpy import array, percentile
from tqdm import tqdm

# Read csv

clusters_and_centers:DataFrame = read_csv(S3_SELECTED_CLUSTERS)

In [4]:

triangle_centers = clusters_and_centers[['centroid_x','centroid_y','centroid_z']].as_matrix()

# clustering with MeanShift
from sklearn.cluster import MeanShift
ms = MeanShift(bandwidth=1, bin_seeding=True)
ms.fit(
    clusters_and_centers[['centroid_x','centroid_y','centroid_z']].as_matrix()
)

MeanShift(bandwidth=1, bin_seeding=True, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [7]:
from sklearn.cluster import AgglomerativeClustering
agc = AgglomerativeClustering(n_clusters=20)
agc.fit(
    clusters_and_centers[['centroid_x','centroid_y','centroid_z']].as_matrix()
)

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=None, linkage='ward', memory=None, n_clusters=20,
            pooling_func=<function mean at 0x7f581418b1e0>)

In [11]:
agc.children_

array([[  159,   160],
       [ 5843,  6039],
       [ 5867,  6061],
       ...,
       [22347, 22348],
       [22349, 22353],
       [22354, 22355]])